In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import cv2
import glob
import os
!pip install git+https://github.com/facebookresearch/segment-anything.git
    
import torch
torch.cuda.empty_cache()
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
sam = sam_model_registry["vit_b"](checkpoint="/kaggle/input/cv-project/sam_vit_b_01ec64.pth")
sam.to('cuda')
mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.86,
    stability_score_thresh=0.92,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100,  # Requires open-cv to run post-processing
)

import math

from transformers import TrOCRProcessor, VisionEncoderDecoderModel
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-printed')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-large-printed')

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-6nkfowrk
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-6nkfowrk
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36621 sha256=9470b9dee19ce6a65baa392c1a1f25a9739ef6042a32485cfed49354eaa71d89
  Stored in directory: /tmp/pip-ephem-wheel-cache-p774b14r/wheels/6e/b2/38/082465c177a067501384fd15cb18be1128372ef60f27a1c426
Successfully built segment-anything


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [ ]:
def rotate_by_angle(img):
    print("Inside rotate_by_angle...\n")
    points = []
    #cropped_img2_2 = img.copy()
    rotated_output = img.copy()
    dst = cv2.Canny(img, 1000, 200, None, 3)
    lines = cv2.HoughLines(dst, 1, np.pi / 180, 150, None, 0, 0)
    # lines = cv2.HoughLinesP(dst, 1, np.pi / 180, 100, minLineLength=100, maxLineGap=10)
    if lines is not None:
        for i in range(0, len(lines)):
            rho = lines[i][0][0]
            theta = lines[i][0][1]
            a = math.cos(theta)
            b = math.sin(theta)
            x0 = a * rho
            y0 = b * rho
            pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
            pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
            points.append((pt1,pt2))
            cv2.line(rotated_output, pt1, pt2, (0,0,255), 3, cv2.LINE_AA)
        if ((pt2[0]-pt1[0])==0):
            angle = 90
        else:
            slope = abs((pt2[1]-pt1[1])/(pt2[0]-pt1[0]))
            #slope = (img.shape[0] - 1 - pt2[1] - img.shape[0] - 1 - pt1[1]) / (pt2[0]-pt1[0]) 
            angle = np.rad2deg(np.arctan(slope))
            print('Slope= ',slope)
            print('angle= ',angle)
#         print(angle)
        rotate_matrix = cv2.getRotationMatrix2D(center=(img.shape[0]/2,img.shape[1]/2), angle=angle, scale=1)
        rotated_image = cv2.warpAffine(src=img, M=rotate_matrix, dsize=(img.shape[0]*2, img.shape[0]*2),flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
        contours, hierarchy = cv2.findContours(cv2.cvtColor(rotated_image, cv2.COLOR_RGB2GRAY), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # get the largest contour
        largest_contour = max(contours, key=cv2.contourArea)

        # get the bounding rectangle of the contour
        x, y, w, h = cv2.boundingRect(largest_contour)

        # crop the image to the bounding rectangle
        crop_img = rotated_image[y:y+h, x:x+w]
#         plt.imshow(angle_line_img)
#         plt.show()
        return (crop_img, rotated_output, np.round(angle,3))
#     return np.zeros(shape=(cropped_img2_2.shape[0]*2,cropped_img2_2.shape[0]*2))
    else:
        print("No lines detecetd")
        return None, None, None

In [ ]:
def get_ocr(img):
    print("Inside get_ocr...\n")
    if img is None:
        return 'None'
    image = img
    pixel_values = processor(images=image, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return(generated_text)

In [ ]:
def segment_books(img, img_path):
    print("Inside segment_books...\n")
    masks = mask_generator.generate(img)
    books_bboxes = []
    boxes_img = img.copy()
    segmentation_output = np.zeros(shape=(img.shape[0],img.shape[1]))
    for i in masks:
        if(i['area']>100000):
            book = {'bbox':None, 'segment':None}
            book['bbox'] = i['bbox']
            book['segment'] = i['segmentation']
            segmentation_output = segmentation_output + i['segmentation']*np.random.randint(10,200)
            books_bboxes.append(book)
            x,y,w,h = i['bbox']
            cv2.rectangle(boxes_img,(int(x),int(y)),((int(x)+int(w)),(int(y)+int(h))),(255,0,0),5)
#     plt.imshow(segmentation_output)
#     plt.imsave('segmentation_output.jpg',segmentation_output)
#     plt.show()
#     plt.imshow(boxes_img)
#     plt.show()
    return books_bboxes, segmentation_output, boxes_img

In [ ]:
def main():
    for image in glob.glob('./input/*'):
        print("Processing ",image)
        img = cv2.imread(image, cv2.COLOR_BGR2RGB)
        img_path = image.split('/')[-1].split('.')[0]
        books_bboxes, segment_output, bounding_box_img = segment_books(img, img_path)
        plt.imshow(segment_output)
        plt.show()
        plt.imshow(bounding_box_img)
        plt.show()
        for i in range(len(books_bboxes)):
            book = books_bboxes[i]
            fig, axes = plt.subplots(1,4,figsize=(8,8))
            x,y,w,h = book['bbox'][0], book['bbox'][1], book['bbox'][2], book['bbox'][3]
            cropped_img = img[int(y):(int(y)+int(h)), int(x):(int(x)+int(w))]
            cropped_mask = book['segment'][int(y):(int(y)+int(h)), int(x):(int(x)+int(w))].astype(int)
            cropped_img2 = np.zeros_like(cropped_img)
            for k in range(3):
                cropped_img2[:,:,k] = np.multiply(cropped_img[:,:,k],cropped_mask)
            rotated_cropped_img, rotation_output, agl = rotate_by_angle(cropped_img2)
            axes[0].imshow(cropped_img2)
            axes[1].imshow(cropped_mask)
            ocr_text = get_ocr(rotated_cropped_img)
            if rotated_cropped_img is None:
                rotated_cropped_img = np.zeros(shape=(cropped_img2.shape[0]*2,cropped_img2.shape[0]*2))
                rotation_output = np.zeros(shape=(cropped_img2.shape[0]*2,cropped_img2.shape[0]*2))
            axes[2].imshow(rotation_output)
            axes[3].set_title("Angle: {}\n{}".format(agl,ocr_text))
            axes[3].imshow(rotated_cropped_img)
            plt.show()
            if not os.path.exists(img_path):
                os.makedirs(img_path)
            plt.imsave("./{}/{}_segmentation.jpg".format(img_path,img_path),segment_output)
            plt.imsave("./{}/{}_boundig_boxes.jpg".format(img_path,img_path),bounding_box_img)
            if not os.path.exists(img_path+'/outputs'):
                os.makedirs(img_path+'/outputs')
            fig.savefig("./{}/{}/{}_{}.jpg".format(img_path, 'outputs',img_path,str(i)+'_output'))
            print("./{}/{}_{}.jpg saved !!!".format(img_path,img_path,str(i)))

    print("Done")


main()


In [ ]:
# import shutil
# # !rm '/kaggle/working/IMG_20230318_134550.zip'
# # !rm '/kaggle/working/IMG_20230318_134552.zip'
# # !rm '/kaggle/working/IMG_20230409_111633.zip'
# # !rm -r '/kaggle/working/IMG_20230318_134550'
# # !rm -r '/kaggle/working/IMG_20230318_134552'
# # !rm -r '/kaggle/working/IMG_20230409_111633'
# # shutil.make_archive('IMG_20230318_134552', 'zip', '/kaggle/working/IMG_20230318_134552')
# shutil.make_archive('IMG_20230318_134550', 'zip', '/kaggle/working/IMG_20230318_134550')
# shutil.make_archive('IMG_20230409_111633', 'zip', '/kaggle/working/IMG_20230409_111633')